<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/Colab/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [9]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [10]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [11]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [25]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p): 
    neighbours = []
    
    for i in range(len(positions[0])):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [17]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [14]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [35]:
def toDictionary(positions, connections):
  edges = []
  for i in range(len(connections[0])):
    for j in range(len(connections[0])):
      if i!=j and i<j and connections[i][j]==1:
        edges.append((i,j,Euclidean_dist(i,j)))
  return edges

In [36]:
class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [45]:
# Initialize graph

graph = Graph()
edges = toDictionary(positions, connections)

for edge in edges:
    graph.add_edge(*edge)

graph.weights

{(0, 1): 19.49822697213286,
 (1, 0): 19.49822697213286,
 (0, 2): 31.57060966238043,
 (2, 0): 31.57060966238043,
 (0, 3): 16.53253063367998,
 (3, 0): 16.53253063367998,
 (0, 4): 27.365905113565425,
 (4, 0): 27.365905113565425,
 (0, 5): 41.64484507236932,
 (5, 0): 41.64484507236932,
 (0, 6): 31.065669006769824,
 (6, 0): 31.065669006769824,
 (0, 7): 49.91916519149325,
 (7, 0): 49.91916519149325,
 (0, 8): 60.821899890312004,
 (8, 0): 60.821899890312004,
 (0, 17): 110.26496976737384,
 (17, 0): 110.26496976737384,
 (0, 18): 105.53494277544257,
 (18, 0): 105.53494277544257,
 (0, 22): 120.29097399751358,
 (22, 0): 120.29097399751358,
 (0, 23): 104.17539782233872,
 (23, 0): 104.17539782233872,
 (0, 24): 85.49794842513104,
 (24, 0): 85.49794842513104,
 (0, 27): 30.7440634806438,
 (27, 0): 30.7440634806438,
 (0, 28): 78.85253876500566,
 (28, 0): 78.85253876500566,
 (0, 29): 72.24658225436669,
 (29, 0): 72.24658225436669,
 (0, 30): 79.0436037438605,
 (30, 0): 79.0436037438605,
 (0, 31): 50.5696493

In [69]:
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    print("Shortest Path Final: ", shortest_paths)
    print(len(shortest_paths))
    print()

    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path

In [70]:
dijsktra(graph, 1, 2)

Shortest Path Final:  {1: (None, 0), 0: (1, 19.49822697213286), 4: (1, 15.344590572116752), 5: (1, 36.720471587727296), 6: (1, 25.798750941939833), 7: (1, 52.988761474508685), 17: (1, 114.70425154607828), 18: (1, 107.84129652866407), 22: (1, 125.6035193958912), 23: (1, 112.0130201522961), 24: (1, 90.88549439891962), 33: (1, 42.197853587912384), 34: (1, 67.55024761276022), 35: (1, 45.286121681769224), 36: (1, 41.395190709541474), 37: (1, 48.96544753911202), 47: (1, 70.24085215312306), 48: (1, 66.83591484552962), 82: (1, 79.30335523867505), 8: (4, 60.511292298086815), 21: (36, 121.8798022779348), 29: (4, 80.98964485287327), 30: (33, 83.588677754895), 31: (4, 58.13297221661141), 46: (35, 100.10646928402869), 49: (6, 94.98735312576386), 2: (0, 51.06883663451329), 3: (0, 36.03075760581284), 27: (0, 50.24229045277666), 28: (33, 88.34910530740797), 38: (0, 46.57641833861075), 40: (0, 58.2580961314209), 9: (5, 69.09378173344342), 16: (5, 84.3524001638865), 39: (36, 68.34693452347906), 44: (3, 

[1, 0, 2]